# Create Endpoint 
# Copy to AWS sagemaker notebook

In [6]:
endpoint_name="falcon-role-1"

In [7]:
#!pip install -U sagemaker # 2.197.0
#!pip install -U sagemaker # 

In [13]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'tiiuae/falcon-7b-instruct',
    'HF_TASK':'text-generation',
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    
    image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
    env=hub,
    role=role,
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    container_startup_health_check_timeout=300,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
---------------!

In [16]:
prompt = "You are a Dungeon Master: Describe the action of an elf called paco trying to jump off a cliff and failing"

payload = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : 256, 
    }
}


import time

ut = time.time()

ans = predictor.predict(payload)

print(ans)

print('sec  time:',time.time() - ut)

[{'generated_text': "You are a Dungeon Master: Describe the action of an elf called paco trying to jump off a cliff and failing\nAs the elf, paco, approached the cliff's edge, he mustered all his strength to jump off. However, as he took that first step forward, his foot slipped on a loose stone, causing him to lose his balance and fall back. He tried to regain his footing, but it was too late. With a loud splash, paco fell into the river below, his body entangled in the rocks."}]
sec  time: 2.781944751739502


In [21]:
%%time

prompt = "You are a Dungeon Master: Describe the action of an elf called paco trying to jump off a cliff and failing"

payload = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : 256, 
    }
}

endpoint_n = endpoint_name
import boto3
runtime = boto3.client("runtime.sagemaker")
response = runtime.invoke_endpoint(EndpointName=endpoint_n, ContentType="application/json", Body=json.dumps(payload))
res = response['Body'].read().decode('utf8')
res = json.loads(res)[0]['generated_text']
print(res)

You are a Dungeon Master: Describe the action of an elf called paco trying to jump off a cliff and failing
As the elf, paco, approached the cliff's edge, he mustered all his strength to jump off. However, as he took that first step forward, his foot slipped on a loose stone, causing him to lose his balance and fall back. He tried to regain his footing, but it was too late. With a loud splash, paco fell into the river below, his body entangled in the rocks.
CPU times: user 29.1 ms, sys: 0 ns, total: 29.1 ms
Wall time: 2.84 s


# To remove from AWS data created.

In [22]:
import boto3
endpoint_n = endpoint_name

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker')#, region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_n)

# Delete endpoint Config
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_n)




{'ResponseMetadata': {'RequestId': '25d3037d-e7ee-4312-a8dd-f4f08b7eb0da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '25d3037d-e7ee-4312-a8dd-f4f08b7eb0da',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 08 Nov 2023 10:53:02 GMT'},
  'RetryAttempts': 0}}